In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import anchor_tabular
from sklearn.model_selection import train_test_split
import re
import copy

In [2]:
import random
import pandas as pd
import matplotlib.pyplot as plt 

In [3]:
## 定義一個 class
class Bunch(object):
    """bla"""
    def __init__(self, adict):
        self.__dict__.update(adict)
dataset = Bunch({})

In [4]:
# 原本的 code
seed = 42
random.seed(seed)

df = pd.read_csv('BankChurners.csv')
df = df[df.columns[:-2]]
df = df.fillna(0)
df['Attrition_Flag'] = df['Attrition_Flag'].replace({'Attrited Customer':1, 'Existing Customer':0})
df['Gender'] = df['Gender'].replace({'M':1, 'F':0})
df = df.drop('CLIENTNUM',axis=1)
df = pd.get_dummies(data=df, columns=['Education_Level', 'Marital_Status', 
                                      'Income_Category', 'Card_Category']) # one-hot encoding


In [5]:
churn_index = df.index[df.Attrition_Flag == 1].tolist()
non_churn_index = df.index[df.Attrition_Flag == 0].tolist()
non_churn_index = random.sample(non_churn_index, len(churn_index)) # 讓 churn : non_churn 資料個數為 1 : 1
new_idx = churn_index + non_churn_index
df = df.loc[new_idx]

In [6]:
# 把原本的 pandas.DataFrame 轉成 np.array 
y = df['Attrition_Flag'].to_numpy()
df_no_y = df.drop('Attrition_Flag', axis=1)
x = df_no_y.to_numpy()
print(y.shape)
print(x.shape)

(3254,)
(3254, 36)


In [7]:
# 模型預測
c = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=100)
c.fit(train_x, train_y)
print('Train', sklearn.metrics.accuracy_score(train_y, c.predict(train_x)))
print('Test', sklearn.metrics.accuracy_score(test_y, c.predict(test_x)))

Train 1.0
Test 0.9416282642089093


In [8]:
# 定義 anchor 中的參數
dataset.train = train_x # 訓練資料 (numpy)
dataset.test = test_x  # 測試資料 (numpy)
dataset.categorical_names = {} # label encoding前的內容 (dict) 
dataset.class_names = ['Existing Customer','Attrited Customer'] # label的名稱 (list) 
dataset.feature_names = np.array(df_no_y.columns.to_list()) # feature的名稱 (numpy)

In [9]:
# anchor explainer 
explainer = anchor_tabular.AnchorTabularExplainer(
    dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)

In [16]:
'''
input 內容:
    1. start -> 從哪一個客戶開始找 anchor
    2. number -> 要從幾個客戶身上找 anchor
    3. dataset -> 訓練模型時使用的 dataset
    4. df -> 原始表格
    5. model -> 要解釋的模型
    6. seed -> 亂數種子(預設為1) 
'''
def customer_list(start, number, dataset, df, model, seed = 1):

    # 製作 anchor 解釋器
    explainer = anchor_tabular.AnchorTabularExplainer(
    dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)

    # 用　Anchors 找到規則
    anchor_list = []
    current_number = 0
    np.random.seed(seed)
    
    for i in dataset.test[start:]:
        if (current_number == number):
            break
        if(model.predict(i.reshape(1, -1))[0]):
            exp = explainer.explain_instance(i, model.predict, threshold=0.95)
            anchor_list.append(exp.names()) # anchor 結果
            current_number += 1
    

    # 將規則做 regular expression
    identical = 0
    anchors = []
    for i,j in enumerate(anchor_list):
        same_element = False 
        for k in anchor_list[i+1:]:
            if set(j) == set(k):
                    same_element = True
                    break
        if not(same_element):
            str_split = [re.split('( <= | >= | < | > )',_j) for _j in j]
            anchors.append(str_split)

    ## 從找到的規則中尋找潛在客戶
    potential_customer = pd.DataFrame()
    for anchor in anchors:
        idxUnion = 0
        temp_df = df
        for n,i in enumerate(anchor):
            if (i[1] == ' >= '):
                idx = temp_df.loc[:,i[0]] >= float(i[2])
            elif (i[1] == ' <= '):
                idx = temp_df.loc[:,i[0]] <= float(i[2])
            elif (i[1] == ' > '):
                idx = temp_df.loc[:,i[0]] > float(i[2])
            elif (i[1] == ' < '):
                idx = temp_df.loc[:,i[0]] < float(i[2])
            else:
                continue
            
            if n == 0:
                idxUnion = idx
            else:
                idxUnion = idx & idxUnion
            
            temp_df = temp_df[idxUnion]
            idxUnion = idxUnion[idxUnion]
            #print(temp_df.shape)
        else:    
            idx = temp_df.loc[:,'Attrition_Flag'] == 0
            idxUnion = idx & idxUnion
            temp_df = temp_df[idxUnion] 
            #print(temp_df.shape) 
            #idxUnion = idxUnion[idxUnion]
            potential_customer = pd.concat([potential_customer,temp_df], join='outer')

    return(potential_customer)

In [18]:
# anchor 
start = 10
number = 2
print(customer_list(start, number, dataset, c, explainer))


      Attrition_Flag  Customer_Age  Gender  Dependent_count  Months_on_book  \
124                0            55       1                2              42   
8248               0            43       1                1              30   
1541               0            45       1                4              37   
1151               0            44       0                3              36   

      Total_Relationship_Count  Months_Inactive_12_mon  Contacts_Count_12_mon  \
124                          2                       3                      2   
8248                         1                       1                      3   
1541                         5                       2                      4   
1151                         6                       3                      3   

      Credit_Limit  Total_Revolving_Bal  ...  Income_Category_$120K +  \
124        34516.0                 1527  ...                        1   
8248       34516.0                 2045  ...        

potential_customer = pd.DataFrame()
for j in anchor:
    idxUnion = 0
    temp_df = df
    for n,i in enumerate(j):
        if (i[1] == ' >= '):
            idx = temp_df.loc[:,i[0]] >= float(i[2])
        elif (i[1] == ' <= '):
            idx = temp_df.loc[:,i[0]] <= float(i[2])
        elif (i[1] == ' > '):
            idx = temp_df.loc[:,i[0]] > float(i[2])
        elif (i[1] == ' < '):
            idx = temp_df.loc[:,i[0]] < float(i[2])
        else:
            continue
        
        if n == 0:
            idxUnion = idx
        else:
            idxUnion = idx & idxUnion
        
        temp_df = temp_df[idxUnion]
        idxUnion = idxUnion[idxUnion]
        #print(temp_df.shape)
    else:    
        idx = temp_df.loc[:,'Attrition_Flag'] == 0
        idxUnion = idx & idxUnion
        temp_df = temp_df[idxUnion] 
        #print(temp_df.shape) 
        #idxUnion = idxUnion[idxUnion]
        potential_customer = pd.concat([potential_customer,temp_df], join='outer')
    

In [12]:
print(potential_customer)

      Attrition_Flag  Customer_Age  Gender  Dependent_count  Months_on_book  \
124                0            55       1                2              42   
8248               0            43       1                1              30   
1541               0            45       1                4              37   
1151               0            44       0                3              36   
3902               0            50       1                3              36   
7931               0            45       1                3              37   
163                0            47       0                4              41   
374                0            55       0                1              43   
3428               0            45       0                3              36   
2136               0            26       0                1              15   
1935               0            44       1                1              33   
113                0            54       0          